# Exercises to Bring it All Together

## Exercise 1 -- Demonstrate Diagram Duality in 2D

For the random set of 2D input points selected below, demonstrate visually that the Delaunay triangulation and Voronoi diagram are duals. Specifically, you should be able to demonstrate that the circumcenters of the circumcircles of the Delaunay triangulation can be connected to form the edges of the Voronoi diagram.

The initial setup is below.

In [ ]:
import numpy as np
import scipy
import scipy.spatial
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

np.random.seed(123)
generators = np.random.randn(15, 2)
generators.shape

Step 1: Plot the generators (input points)

Step 2: Plot the Voronoi diagram along with its generators

Step 3: Plot the Delaunay triangulation alongside the Voronoi diagram and generators 

Step 4: Plot the circumcircles and their centers over top of the Delaunay Triangulation & Voronoi diagram + generators

Step 5: Connect the circumenters of circumcircles of neighboring triangles (hint: you may need to check the SciPy documentation for `Delaunay`--try looking for information about neighboring triangles)

## Exercise 2 -- Algorithm Time Complexity

A common concern in computational geometry is the time complexity of an algorithm (the growth of code execution time as a function of algorithm input size).

Compare the execution times of `numpy.sum()` and `scipy.spatial.Voronoi()` for the same input data sets, collecting the results in NumPy arrays and reporting average and standard deviation values of the execution times over a set of trials with increasingly large data sets.

Step 1: Initialize some NumPy arrays for storing the results of the timings for sum and Voronoi

Step 2: Write a for loop that runs the timing trials and stores the timing results in the NumPy arrays that are appropriately named for the data inputs being stored within them. This will also involve generating matching data sets that work with both sum() and Voronoi()

Step 3: Calculate the average and standard deviation of the results and compare the increases in execution time for sum() and Voronoi() as data input size increases in a plot. Include the standard deviation "error bars" in your plot. Hint: matplotlib has an `errorbar` plot function, for example.